In [29]:
import mphys as mp
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import cm
import matplotlib as mpl
import seaborn as sns
import scipy
from datetime import datetime as dt
import datetime
from IPython.display import Image, display, clear_output
from IPython.core.display import HTML
# from PIL import Image

years = [2, 11]  # The range of years to get
months = [7, 13]  # Months within year to get
dateRange = []
for y in range(*years):
    for m in range(*months):
        dateRange.append([y, m])  # Make list of months & years

with mp.Timer('Timing code'):
    moments = pd.read_csv('../../../moments.csv',
                          index_col=0, parse_dates=True)
    # moments = mp.moments(dateRange)  # Regen moments data. Takes > 90s
    # moments.to_csv('moments.csv')
    # omni = mp.omni(dateRange)
#     pgp = mp.pgp(dateRange, gsm=True)  # Get predicted geometric position
    pgp = pd.read_csv('../../../pgp.csv', index_col=0, parse_dates=True)
    omni = pd.read_csv('../../../omni.csv', index_col=0, parse_dates=True)

    # data = pd.merge_asof(moments, omni, left_index=True, right_index=True)
    data = pd.merge_asof(moments, pgp, left_index=True, right_index=True)
    data = pd.merge_asof(data, omni, left_index=True, right_index=True)
    RE = 6371  # km
    data.z = data.z / RE  # Units to RE
    data.x = data.x / RE
    data.y = data.y / RE

    # plot conifg
    # fig = plt.figure(figsize=[6, 6])
    # plt.rc('font', family='serif')
    # plt.rc('xtick', labelsize='x-small')
    # plt.rc('ytick', labelsize='x-small')

    T = 20  # Temperature lower limit

    data = data[(data.z > 7) | (data.z < -7)]
    data = data[data.index > dt(2005, 1, 1, 0, 0, 0)]
    # https://stackoverflow.com/q/24281936
    data['tag'] = data['temp'] > T
    fst = data.index[data.tag & ~ data.tag.shift(1).fillna(False)]
    lst = data.index[data.tag & ~ data.tag.shift(-1).fillna(False)]

    fmtUrl = 'https://ssusi.jhuapl.edu/dataN/f16/apl/l1b/images/{0}/f16_StripDisk_{1}.png'
    dates = [[fmtUrl.format(i.strftime("%Y/%j"),
                            i.strftime("%Y%j")),
              f"{i.strftime('%Y-%m-%d %H:%M:%S')} for {str(j-i)}",
              i, j] for i, j in zip(fst, lst)
             if ((j >= i + pd.DateOffset(hours=1.5)) & (j <= i + pd.DateOffset(days=1)))]
    imf_filt = []
    for d in range(len(dates)):
        test = data[(data.index >= dates[d][2]) & (data.index <= dates[d][3])]
        if test.bz.mean() > 0:
            imf_filt.append(dates[d] + [test.bz.mean()])
    clear_output(wait=True)
    doesHaveArc = []
    print(imf_filt[:10])
    with open('arcs.csv', 'w') as f:
        f.write('start,end,imf_z,arc,url\n')
    for i, d in enumerate(imf_filt[:10]):
        print(f'displaying {i}/{len(imf_filt)}')
        display(Image(url = d[0]))
        print(d[1])
        doesHaveArc.append(input('Have Arc? '))
        if doesHaveArc[-1] == 'y':
            doesHaveArc[-1] = True
        else:
            doesHaveArc[-1] = False
        stuff = ','.join([imf_filt[i][2].strftime('%Y-%m-%d %H:%M:%S'),
                imf_filt[i][3].strftime('%Y-%m-%d %H:%M:%S'),
                str(imf_filt[i][4]),
                str(doesHaveArc[-1]),
                dates[i][0]])+'\n'
        with open('arcs.csv', 'a') as f:
            f.write(stuff)
        clear_output(wait=True)


Timing code completed in 138.36s
